# 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [1]:
import requests
from bs4 import BeautifulSoup
import re   #將網站轉換成 編碼後的url
import os
from urllib.request import urlretrieve 

# 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [2]:
input_keyword="網路爬蟲"  # define the word we wanna search

url_utf8=repr(input_keyword.encode("UTF-8")).upper()  #representation # 編碼成UTF-8並轉成大寫字元
url_utf8 = url_utf8.replace("\\X", "%") #replace \x to %
print(url_utf8)
print("%s: %s" % (input_keyword, url_utf8[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + url_utf8[2:-1:1]   # 家一下
print(root_keyword_link)


'''
1. repr
其產生的字串是給python的直譯器看的，這個字串會顯示'明確且教詳盡的資訊'
通常 可以讓python得知究竟這字串所代表的對象為何
2.幹 其實根本不用轉換
'''

B'%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2'
網路爬蟲: %E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2
/wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2


"\n1. repr\n其產生的字串是給python的直譯器看的，這個字串會顯示'明確且教詳盡的資訊'\n通常 可以讓python得知究竟這字串所代表的對象為何\n2.幹 其實根本不用轉換\n"

# 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [9]:
# 以防萬一 加個 headers
headers={
        'authority': 'zh.wikipedia.org' , 
'method': 'GET' ,
'path':  root_keyword_link      # 好像要換掉喔
}

url = "https://zh.wikipedia.org" + root_keyword_link 
resp=requests.get(url , headers=headers)
resp.encoding='UTF-8'
html=BeautifulSoup(resp.text , "lxml")

In [10]:
#爬取該關鍵字的文章內容
content=html.find('div',attrs={"id":"mw-content-text"}).find_all("p")
#print(content.get_text())   it does not work cause it needs A string to convert I guess
for paragraph in content:
    print(paragraph.text)

网络爬虫（英語：web crawler），也叫網路蜘蛛（spider），是一种用来自动浏览万维网的网络机器人。其目的一般为编纂网络索引（英语：Web indexing）。

網路搜索引擎等站点通过爬蟲軟體更新自身的網站內容（英语：Web content）或其對其他網站的索引。網路爬蟲可以將自己所訪問的頁面保存下來，以便搜索引擎事後生成索引（英语：Index (search engine)）供用戶搜索。

爬蟲访问网站的过程会消耗目标系统资源。不少网络系统并不默许爬虫工作。因此在访问大量页面时，爬虫需要考虑到规划、负载，还需要讲“礼貌”。 不愿意被爬虫访问、被爬虫主人知晓的公开站点可以使用robots.txt文件之类的方法避免访问。这个文件可以要求机器人（英语：Software agent）只对网站的一部分进行索引，或完全不作处理。

互联网上的页面极多，即使是最大的爬虫系统也无法做出完整的索引。因此在公元2000年之前的万维网出现初期，搜索引擎经常找不到多少相关结果。现在的搜索引擎在这方面已经进步很多，能够即刻给出高质量结果。

爬虫还可以验证超連結和HTML代码，用于网络抓取（英语：Web scraping）（参见数据驱动编程）。

网络爬虫也可称作网络蜘蛛[1]、蚂蚁、自动索引程序（automatic indexer）[2] ，或（在FOAF（英语：FOAF (software)）软件中）称为网络疾走（web scutter）。[3]

网络爬虫始于一张被称作种子的统一资源地址（URL）列表。当网络爬虫访问这些统一资源定位器时，它们会甄别出页面上所有的超链接，并将它们写入一张“待访列表”，即所谓爬行疆域（英语：crawl frontier）。此疆域上的URL将會被按照一套策略循环來访问。如果爬虫在执行的过程中复制归档和保存网站上的信息，这些档案通常储存，使他们可以較容易的被查看。阅读和浏览他们儲存的网站上並即时更新的信息，這些被儲存的網頁又被稱為“快照”。越大容量的網頁意味着网络爬虫只能在给予的时间内下载越少部分的网页，所以要优先考虑其下载。高变化率意味着网页可能已经被更新或者被取代。一些服务器端软件生成的URL（统一资源定位符）也使得网络爬虫很难避免检索到重复内容。

但是互联网的资源卷帙浩繁，这也意味着网络爬虫只能在一定时间内下载有限数量的网页，因此它需要衡

# 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [6]:
for ex_link in content:
    ex_tag= ex_link.find_all("a", href=re.compile("^(/wiki/)((?!;)\S)*$"))   #  不建議 .a 一個 <P> 裡可能有好幾個
#    print(ex_tag) 
#    test=ex_link.find_all("a", href=re.compile("^(/wiki/)"))   
#    print(test)
#  效果同上
    for  link_string in ex_tag:
         print("外部連結:" ,link_string['title'] , link_string['href'])
'''
1.這邊  如何 擷取 a的屬性是 重點
因為 href 會變化 需要用到 正規表達式
2.("^(/wiki/)((?!;)\S)*$")
^ 表示  取以 xxx 為開頭的 string  不過其實有許多意思    例如：/^A/ 不會匹配「an A」的 A，但會匹配「An E」中的 A。  
$  ''   取以 xxx 結尾的 string /t$/ 不會匹配「eater」中的 t，卻會匹配「eat」中的 t。  
\S      不讀取 空白以後的東西
(?!;)   後面不是 ; 的    ^(/wiki/)

'''

外部連結: 万维网 /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91
外部連結: 网络机器人 /wiki/%E7%BD%91%E7%BB%9C%E6%9C%BA%E5%99%A8%E4%BA%BA
外部連結: 网络搜索引擎 /wiki/%E7%BD%91%E7%BB%9C%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結: 搜索引擎 /wiki/%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結: Robots.txt /wiki/Robots.txt
外部連結: 网站 /wiki/%E7%BD%91%E7%AB%99
外部連結: 超連結 /wiki/%E8%B6%85%E9%80%A3%E7%B5%90
外部連結: HTML /wiki/HTML
外部連結: 数据驱动编程 /wiki/%E6%95%B0%E6%8D%AE%E9%A9%B1%E5%8A%A8%E7%BC%96%E7%A8%8B
外部連結: 網頁 /wiki/%E7%B6%B2%E9%A0%81
外部連結: 互联网 /wiki/%E4%BA%92%E8%81%94%E7%BD%91
外部連結: 服务器 /wiki/%E6%9C%8D%E5%8A%A1%E5%99%A8
外部連結: 超文本傳輸協定 /wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E5%82%B3%E8%BC%B8%E5%8D%94%E5%AE%9A


'\n1.這邊  如何 擷取 a的屬性是 重點\n因為 href 會變化 需要用到 正規表達式\n2.("^(/wiki/)((?!;)\\S)*$")\n^ 表示  取以 xxx 為開頭的 string  不過其實有許多意思    例如：/^A/ 不會匹配「an A」的 A，但會匹配「An E」中的 A。  \n$  \'\'   取以 xxx 結尾的 string /t$/ 不會匹配「eater」中的 t，卻會匹配「eat」中的 t。  \n\\S      不讀取 空白以後的東西\n(?!;)   後面不是 ; 的    ^(/wiki/)\n\n'

# 大師程式

In [7]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [万维网] /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91
外部連結: [网络机器人] /wiki/%E7%BD%91%E7%BB%9C%E6%9C%BA%E5%99%A8%E4%BA%BA
外部連結: [網路] /wiki/%E7%BD%91%E7%BB%9C%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結: [搜索引擎] /wiki/%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結: [robots.txt] /wiki/Robots.txt
外部連結: [网站] /wiki/%E7%BD%91%E7%AB%99
外部連結: [超連結] /wiki/%E8%B6%85%E9%80%A3%E7%B5%90
外部連結: [HTML] /wiki/HTML
外部連結: [数据驱动编程] /wiki/%E6%95%B0%E6%8D%AE%E9%A9%B1%E5%8A%A8%E7%BC%96%E7%A8%8B
外部連結: [網頁] /wiki/%E7%B6%B2%E9%A0%81
外部連結: [互联网] /wiki/%E4%BA%92%E8%81%94%E7%BD%91
外部連結: [服务器] /wiki/%E6%9C%8D%E5%8A%A1%E5%99%A8
外部連結: [超文本傳輸協定] /wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E5%82%B3%E8%BC%B8%E5%8D%94%E5%AE%9A


# 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
(1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)

(2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字

(3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [24]:
def WikiArticle(key_word_link, key_word, recursive):
     if (recursive <= max_recursive_depth):
        print("遞迴層[%d] -%s (%s)" % (recursive, key_word_link, key_word)) 
        
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    
        '''
        ---------------以下為查詢關鍵字與擷取,儲存內容------------------
        '''
        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
       # out_dir='WikiArticle'
        if not os.path.exists('./WikiArticle/'):
            os.makedirs( './WikiArticle/')   #os.makedirs( './Data' ,exist_ok =True )
    #    with open ('./WikiArticle/WikiArticle.txt' , 'w' , encoding='utf-8') as  data:
        key_word=key_word.replace("/", "_");
    #    print(key_word)
        f = open('./WikiArticle/'+key_word+'.txt', "w",encoding="utf-8")    
        for string in content:
                f.write(string.text)
        f.close()
        '''
        --------------儲存成dict 以便使用 recursive -----------
        1.特別注意 key_word=key_word.replace("/", "_");
        檔名的設定要特別注意
        不能包括
        \/>?  ..  etc.
        '''
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        for i in content:
            a_tag=i.find_all( ("a") , href=re.compile("^/wiki/"))
     #       if len(a_tag) > 0:
            for ex_url in a_tag:
                    ex_link=ex_url['href']
                    ex_title=ex_url['title']
                    external_link_dict[ ex_link ] = ex_title
              #  print(ex_title , ":","https://zh.wikipedia.org"+ex_link)
#        data.close()
       # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():   # 只是把  "" : " "拿掉((  K , v  只是隨便的變數 這邊用上了ex_link_dict中的數據
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程 '''

In [23]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 1

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] -/wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2 (網路爬蟲)
遞迴層[1] -/wiki/%E4%B8%87%E7%BB%B4%E7%BD%91 (全球資訊網)
遞迴層[1] -/wiki/%E7%BD%91%E7%BB%9C%E6%9C%BA%E5%99%A8%E4%BA%BA (網路機器人)
遞迴層[1] -/wiki/%E7%BD%91%E7%BB%9C%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E (網路搜尋引擎)
遞迴層[1] -/wiki/%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E (搜尋引擎)
遞迴層[1] -/wiki/Robots.txt (Robots.txt)
遞迴層[1] -/wiki/%E7%BD%91%E7%AB%99 (網站)
遞迴層[1] -/wiki/%E8%B6%85%E9%80%A3%E7%B5%90 (超連結)
遞迴層[1] -/wiki/HTML (HTML)
遞迴層[1] -/wiki/%E6%95%B0%E6%8D%AE%E9%A9%B1%E5%8A%A8%E7%BC%96%E7%A8%8B (資料驅動編程)
遞迴層[1] -/wiki/%E7%B6%B2%E9%A0%81 (網頁)
遞迴層[1] -/wiki/%E4%BA%92%E8%81%94%E7%BD%91 (網際網路)
遞迴層[1] -/wiki/%E6%9C%8D%E5%8A%A1%E5%99%A8 (伺服器)
遞迴層[1] -/wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E5%82%B3%E8%BC%B8%E5%8D%94%E5%AE%9A (超文字傳輸協定)
